# Using the GPTMilvusIndex

In this notebook we are going to show a quick demo of using the MilvusIndex. 

In [1]:
import logging
import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from gpt_index import GPTMilvusIndex, SimpleDirectoryReader, Document
from IPython.display import Markdown, display
import textwrap

/Users/filiphaltmayer/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup OpenAI
Lets first begin by adding the openai api key. This will allow us to access openai for embeddings and to use chatgpt.

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

### Generate our data
With our LLM set, lets start using the Milvus Index. As a first example, lets generate a document from the file found in the `paul_graham_essay/data` folder. In this folder there is a single essay from Paul Graham titled `What I Worked On`. To generate the documents we will use the SimpleDirectoryReader.

In [3]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()
print('Document ID:', documents[0].doc_id, 'Document Hash:', documents[0].doc_hash)

Document ID: 933666c4-2833-475a-a3d5-d279a0c174fa Document Hash: 77ae91ab542f3abb308c4d7c77c9bc4c9ad0ccd63144802b7cbe7e1bb3a4094e


### Create an index across the data
Now that we have a document, we can can create an index and insert the document. For the index we will use a GPTMilvusIndex. GPTMilvusIndex takes in a few arguments:

- collection_name (str, optional): The name of the collection where data will be stored. Defaults to "llamalection".
- index_params (dict, optional): The index parameters for Milvus, if none are provided an HNSW index will be used. Defaults to None.
- search_params (dict, optional): The search parameters for a Milvus query. If none are provided, default params will be generated. Defaults to None.
- dim (int, optional): The dimension of the embeddings. If it is not provided, collection creation will be done on first insert. Defaults to None.
- host (str, optional): The host address of Milvus. Defaults to "localhost".
- port (int, optional): The port of Milvus. Defaults to 19530.
- user (str, optional): The username for RBAC. Defaults to "".
- password (str, optional): The password for RBAC. Defaults to "".
- use_secure (bool, optional): Use https. Defaults to False.
- overwrite (bool, optional): Whether to overwrite existing collection with same name. Defaults to False.


In [4]:
# Create an index over the documnts
index = GPTMilvusIndex.from_documents(documents, overwrite=True)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17617 tokens


# Query the data
Now that we have our document stored in the index, we can ask questions against the index. The index will use the data stored in itself as the knowledge base for chatgpt.

In [5]:
response = index.query("What did the author learn?", )

INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 4028 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 6 tokens


In [6]:
print(textwrap.fill(str(response), 100))

  The author learned that working on things that are not prestigious can be a good thing, as it can
lead to discovering something real and avoiding the wrong track. The author also learned that
ignorance can be beneficial, as it can lead to discovering something new and unexpected. The author
also learned the importance of working hard, even at the parts of the job they don't like, in order
to set an example for others. The author also learned the value of unsolicited advice, as it can be
beneficial in unexpected ways, such as when Robert Morris suggested that the author should make sure
Y Combinator wasn't the last cool thing they did.


In [7]:
response = index.query("What was a hard moment for the author?")

INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 4072 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 9 tokens


In [8]:
print(textwrap.fill(str(response), 100))

 A hard moment for the author was when he was dealing with urgent problems during YC and about 60%
of them had to do with Hacker News, a news aggregator he had created. He was overwhelmed by the
amount of work he had to do to keep Hacker News running, and it was taking away from his ability to
focus on other projects. He was also haunted by the idea that his own work ethic set the upper bound
for how hard everyone else worked, so he felt he had to work very hard. He was also dealing with
disputes between cofounders, figuring out when people were lying to them, and fighting with people
who maltreated the startups. On top of this, he was given unsolicited advice from Robert Morris to
make sure Y Combinator wasn't the last cool thing he did, which made him consider quitting.


We can also save the configuration of the index for later use. 

In [9]:
saved_index = index.save_to_dict()

This next test shows that overwriting removes the previous data.

In [15]:
index = GPTMilvusIndex.from_documents([Document("The answer is ten.")], overwrite=True)
res = index.query("Who is the author?")

print(flush=True)
print("Res:", res, flush=True)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 5 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 44 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 5 tokens



Res: 
The author is unknown.


The next test shows reloading the index and also shows adding additional data to an already existing  index.

In [11]:
del index

index = GPTMilvusIndex.load_from_dict(saved_index)
print(index.query("What is the answer."))

index = GPTMilvusIndex.from_documents(documents, overwrite = False)
print(index.query("What is the answer?"))
print(index.query("Who is the author?"))


INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 44 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 5 tokens



The answer is ten.


INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17617 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 41 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 5 tokens



Ten.


INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 3720 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 5 tokens




The author of the text is Paul Graham, co-founder of Y Combinator.
